# Error Analysis

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Content

This tutorial will use the example of a Cross Resonance (CR) gate to demonstrate how to analyze errors that arise during a quantum operation, using tools including dynamical analysis to understand the state evolution, and the truth table to visualize the leakage errors outside of the computational space. The outline of this tutorial is as follows:

- Preparation
- Introduction
- Dynamical analysis
- Truth table
- Summary

## Preparation

After you have successfully installed Quanlse on your device, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# This module is imported for creating Hamiltonian dictionary
from Quanlse.Utils import Hamiltonian as qham

# These functions are imported to define useful operators matrices to free us from defining them manually
from Quanlse.Utils.Operator import number, duff, dagger, driveX, driveY, basis, projector

# These functions are imported to helps us perform matrix calculation
from Quanlse.Utils.Tools import tensor, project

# This function is imported to perform CR gate optimization using Quanlse Cloud Service
from Quanlse.remoteOptimizer import remoteOptimizeCr

# This module is imported to define frequently-used matrix form for quantum gates
from Quanlse.QOperation import FixedGate

# This module is imported to perform figure plotting
from Quanlse.Utils import Plot

import numpy
import matplotlib.pyplot as plt

To use `remoteOptimizeCr()` function, we need a token to get access to Quanlse Cloud Service.

In [ ]:
# Import Define class and set the token
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

## Introduction

When a target quantum gate is implemented using control pulses, the actual quantum operation might differ from the target operation. This difference is normally quantified as infidelity. Understanding the sources of errors that lead to this infidelity is important for us to design pulse sequences that can mitigate these errors and improve the gate fidelity. 

Quanlse provides the functionality to analyze the population evolution of qubit states through dynamical analysis, and visualize the leakage out of the computational space through a truth table. In this tutorial, we will demonstrate these two functions using the implementation of Cross Resonance (CR) gate as an example. More details on CR gate can be found in [Cross Resonance gate](https://quanlse.baidu.com/#/doc/tutorial-cr).

Using the following lines of code, we first construct our Hamiltonian and optimize the control parameters to implement a high-fidelity CR gate.

In [ ]:
# Define parameters to initialize the Hamiltonian
dt = 2.0  # Sampling period
qubits = 2  # Number of qubits
level = 3  # Energy level

# Define qubit parameters
g = 0.0038 * (2 * numpy.pi)  # Coupling strength, GHz
wq0  = 4.914 * (2 * numpy.pi)  # Transition frequency for qubit 0, GHz
wq1 = 4.714 * (2 * numpy.pi)  # Transition frequency for qubit 1, GHz
wd1 = wq1  # Drive frequency is the frequency for qubit 1
anharm0 = - 0.33 * (2 * numpy.pi)  # Anharmonicity of qubit 0, GHz
anharm1 = - 0.33 * (2 * numpy.pi)  # Anharmonicity of qubit 1, GHz

# Initialize the Hamiltonian
ham = qham.createHam(title="cr-gate", dt=dt, qubitNum=qubits, sysLevel=level)

# Add the detuning terms
term_detuning0 = (wq0 - wd1) * number(level)
term_detuning1 = (wq1 - wd1) * number(level)

qham.addDrift(ham,name='detuning0', onQubits=0, matrices=term_detuning0)
qham.addDrift(ham,name='detuning1', onQubits=1, matrices=term_detuning1)

# Add the anharmonicity terms
qham.addDrift(ham, name='anharm0', onQubits=0, matrices=duff(level), amp=anharm0 / 2)
qham.addDrift(ham, name='anharm1', onQubits=1, matrices=duff(level), amp=anharm1 / 2)

# Add the coupling term
qham.addCoupling(ham, "coupling", [0, 1], g=g / 2)

# Add the control terms
qham.addControl(ham, name="q0-ctrlx", onQubits=0, matrices=driveX(level))
qham.addControl(ham, name="q0-ctrly", onQubits=0, matrices=driveY(level))

# Set amplitude bound
aBound = (1.0, 3.0)

# Run the optimization on Quanlse Cloud Service
ham, infidelity = remoteOptimizeCr(ham, aBound=aBound, tg=200, maxIter=3, targetInfidelity=0.01)

## Dynamical analysis

Studying how the initial qubit state evolves during a quantum operation helps us to understand the effect of control pulses on the qubit and the sources of errors. The dynamical analysis functionality in Quanlse simulates the population evolution of different qubit states for a given initial state. In the following example, we demonstrate how to simulate the population evolution of basis states in the computational space for a two-qubit system ($|00\rangle$, $|01\rangle$, $|10\rangle$ and $|11\rangle$) when a CR gate is implemented. We will look at the population evolution of basis states when the initial qubit state is $|01\rangle$ as an example.

We first define our complete set of computational basis states for a two-qubit system by firstly generating a state vector for each qubit using `basis` function, which takes the number of energy levels (3 for a three-level system) as the first input, and the state (0 or 1 in this case) as the second input. The state vectors representing these two qubits are then constructed from each state vector using `tensor` function.

In [ ]:
# Define basis states: 00, 01, 10 and 11
state00 = tensor([basis(3, 0), basis(3, 0)])
state01 = tensor([basis(3, 0), basis(3, 1)])
state10 = tensor([basis(3, 1), basis(3, 0)])
state11 = tensor([basis(3, 1), basis(3, 1)])
stateList = [state00, state01, state10, state11]

We then construct the projection matrices to calculate the population for each basis state from the expectation value of each projection matrix. We use `projector` function to create the projection matrix. `projector(a, b)` takes two previously defined state vetors (a and b) and generates the projection matrix $|a\rangle\langle b|$. If only one state vector is taken as the input, for example, $|a\rangle$, this function will return $|a\rangle\langle a|$.

In [ ]:
# Construct projection matrices from basis states
matrix00 = projector(state00)
matrix01 = projector(state01)
matrix10 = projector(state10)
matrix11 = projector(state11)
matrixList = [matrix00, matrix01, matrix10, matrix11]

We are now ready to evaluate the evolution of the expectation value for each projection matrix during the CR gate operation. We use the function `Benchmark.evolution` to do so. This function takes the following inputs: the constructed Hamiltonian, the list of initial state vectors and the list of projection matrices.

In [ ]:
# Run the simulation to evaluate evolution of the expectation values
from Quanlse.Utils import Benchmark
evolutionResult = Benchmark.evolution(ham, stateInitial=stateList, matrix=matrixList)

The returned **result** is a dictionary containing various terms. The keys *'0'*, *'1'*, *'2'* … refer to the indices of the initial states. Corresponding to each key for state index, there is a sub-dictionary that contains the following terms:

* 'state_form': the initial state vector
* 'state_evolution_history': the evolution of the initial state vector
* 'result': the evolution of the expectation values for the projection matrices corresponding to different states. Within this sub-dictionary, it contains the expectation value for each projection matrix.

Now we can plot the population evolution of different basis states by taking the absolute value of each expectation value. In the following lines, we plot the population evolution as a function of time with an initial state defined to be $|01\rangle$.

In [ ]:
# Define x values to be the time for evolution
x = numpy.linspace(0, ham['circuit']['max_time_ns'], ham['circuit']['max_time_dt'])

# Define y values to be the expectation value for each projection matrix when the initial state is in 01, which corresponds to index 1
y1 = numpy.array(evolutionResult['1']['result']['matrix-0-value'])
y2 = numpy.array(evolutionResult['1']['result']['matrix-1-value'])
y3 = numpy.array(evolutionResult['1']['result']['matrix-2-value'])
y4 = numpy.array(evolutionResult['1']['result']['matrix-3-value'])

# Plot the population as absolute value of the expectation values
plt.plot(x, abs(y1), linewidth=3, label='00')
plt.plot(x, abs(y2), linewidth=3, label='01')
plt.plot(x, abs(y3), linewidth=3, label='10')
plt.plot(x, abs(y4), linewidth=3, label='11')
plt.title(r'Population Evolution for $|01\rangle$ Initial State ')
plt.xlabel('Time (ns)')
plt.ylabel('Population')
plt.legend()
plt.show()

## Truth table

A truth table of a quantum gate contains the probability of the system being in each possible basis states at the end of an operation for each possible initial state. Initial states are always selected to be in the computational space, while the final states can be outside of the computational space, which correponds to a leakage event. Quanlse provides the functionality to calculate the elements of the truth table for a quantum operation and visualize the results using a 2D plot. This is a convenient tool for us to analyze the errors corresponding to the leakage for all initial basis states.

Here, we will introduce this tool by analyzing the unitary operator of the CR gate, which is generated in the *Introduction* section.

In [ ]:
# Import the function for generating the truth table
from Quanlse.Utils.Benchmark import stateTruthTable

# Import the function for plotting the heat map
from Quanlse.Utils.Plot import plotHeatMap

# Import the functions for basis operations
from Quanlse.Utils.Tools import generateBasisIndexList, computationalBasisList

# Indicate the input state list, and generate the list of the state indices
inputStateStr = ['00', '01', '10', '11']
initStateList = generateBasisIndexList(inputStateStr, level)

# Generate the matrix of the truth table
result = qham.simulate(ham)
matrix = stateTruthTable(result["unitary"], qubits, level, initStateList)

# Generate the list of the output state strings and plot the heat map
outputStateStr = computationalBasisList(qubits, level)
plotHeatMap(matrix, xTicks=outputStateStr, yTicks=inputStateStr, xLabel="Output State", yLabel="Input State", useLog=True)

From the above heat map, we can easily read the information about the evolution, and the energy level leakage. 

## Summary

In summary, this tutorial illustrated the Quanlse functionalities for analyzing errors that might arise when a quantum gate is implemented through control pulses. Dynamical analysis gives us a complete picture of the state evolution, and truth table informs us the distribution of errors on different states.

After reading this tutorial on error analysis, the users could follow this link [tutorial-error-analysis.ipynb](https://github.com/baidu/Quanlse/tree/master/Tutorial/EN/tutorial-error-analysis.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to explore other advanced research techniques that are not shown in this tutorial.

## References
\[1\] Nielsen, Michael A., and Isaac L. Chuang. *Quantum Computation and Quantum Information: 10th Anniversary Edition*. Cambridge: Cambridge UP, 2010. Print.
